In [ ]:
%%capture
# Installs Unsloth, Xformers (for speed), and TRL (for training loops)
!pip install unsloth
# Also install the latest nightly version for GGUF export support
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

# 1. Configuration for Summarization (High Context, Low Memory)
max_seq_length = 16384  # Increased to 16k to read long legal documents
dtype = None            # Auto-detect
load_in_4bit = True     # Essential for running 16k context on T4

# 2. Load the Qwen3-0.6B (The "Extractor" Model)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters (High Capacity)
# We use r=64 because 0.6B is small; it needs more trainable parameters to master summarization.
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,               # Higher rank for complex task
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 128,     # Alpha = 2 * Rank (Aggressive updates)
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ Qwen3-0.6B Summarizer loaded with 16k context!")

In [ ]:
from datasets import load_dataset

# 1. Load the file
dataset = load_dataset("json", data_files="barq_2k_finetune.jsonl", split="train")

# 2. Define the QFS Prompt Template
# This forces the model to ignore chat and focus on extraction.
summarization_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Query-Focused Summarization engine. Extract relevant information from the provided document to answer the user query. Output ONLY the summary. Do not use internal reasoning or conversational filler.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    # Mapping old "conversations" format to QFS format
    # Input = Document + Query (User Message)
    # Output = Summary (Assistant Message)
    inputs = [c[1]["content"] for c in examples["conversations"]]
    outputs = [c[2]["content"] for c in examples["conversations"]]

    texts = []
    for input_text, output_text in zip(inputs, outputs):
        # CRITICAL: Remove <think> blocks to train for "Non-Thinking" Mode
        if "</think>" in output_text:
            clean_output = output_text.split("</think>")[-1].strip()
        else:
            clean_output = output_text

        # Format: Input contains the doc & query, Output contains the summary
        text = summarization_prompt.format(input_text, clean_output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# 3. Apply the format
dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. Verify
print("\n🔍 Training Data Sample (Non-Thinking Summarization):")
print("="*60)
print(dataset[0]["text"][:500] + "...")

In [ ]:
import psutil
import builtins

# Patch the missing library into the system internals
builtins.psutil = psutil

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Set to False for Instruction Tuning
    args = TrainingArguments(
        per_device_train_batch_size = 1, # Keep low for 16k context
        gradient_accumulation_steps = 8, # Accumulate to get stable updates
        warmup_steps = 50,
        max_steps = 750, # Train for roughly 3 epochs on 2k data
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 25,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "barq_summarizer_outputs",
    ),
)

print("🚀 Starting Summarization Training...")
trainer.train()
print("✅ Done! The 'Thinking' has been removed.")

In [ ]:
FastLanguageModel.for_inference(model)

# 1. Define a Context (Simulating a retrieved document)
document = """
المادة 50: يمنع تجاوز الإشارة الحمراء. الاستثناء الوحيد هو لمركبات الطوارئ الرسمية (إسعاف، إطفاء) عند تشغيل المنبهات.
المادة 51: يجب على السائقين إفساح الطريق لمركبات الطوارئ.
"""
query = "متى يسمح بقطع الإشارة؟"

# 2. Format Input
# We combine Doc + Query into the 'Input' field
input_text = f"DOCUMENT:\n{document}\n\nQUERY:\n{query}"

# 3. Use the Training Template
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Query-Focused Summarization engine. Extract relevant information from the provided document to answer the user query. Output ONLY the summary. Do not use internal reasoning or conversational filler.

### Input:
{}

### Response:
""".format(input_text)

# 4. Generate
inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,  # Short summary
    use_cache = True,
    temperature = 0.1,     # Low temp for factual extraction
)

print("🤖 Summary:")
print(tokenizer.batch_decode(outputs)[0].split("### Response:")[-1].replace("<|endoftext|>", "").strip())

In [ ]:
# Save to GGUF (Mobile Format)
model.save_pretrained_gguf(
    "barq_mobile_v1",
    tokenizer,
    quantization_method = "q4_k_m"
)

In [ ]:
from huggingface_hub import login, HfApi

# 1. سجل دخولك (سيطلب منك Token، احصل عليه من إعدادات حسابك في Hugging Face)
login()

# 2. ارفع الملف
api = HfApi()
repo_id = "y3fai/barq" # استبدل your-username باسمك

# إنشاء المستودع (إذا لم يكن موجوداً)
api.create_repo(repo_id=repo_id, exist_ok=True, repo_type="model")

# رفع الملف
print("⏳ Uploading to Hugging Face...")
api.upload_file(
    path_or_fileobj="qwen3-1.7b.Q4_K_M.gguf", # تأكد من مسار الملف عندك
    path_in_repo="qwen3-1.7b.Q4_K_M.gguf",
    repo_id=repo_id,
    repo_type="model"
)
print("✅ Upload Complete! Your model is safe.")